In [3]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

chunk_size = 100000

filtered_chunks = []

for chunk in pd.read_csv("/content/drive/My Drive/US_Accidents_March23.csv", chunksize=chunk_size):
    filtered_chunk = chunk[chunk['State'] == 'GA']
    filtered_chunks.append(filtered_chunk)


df_ga = pd.concat(filtered_chunks, ignore_index=True)

y = df_ga['Severity']
X = df_ga.iloc[:, 11:]
X.drop(columns=['State', 'Country', 'Timezone', 'Airport_Code'], inplace = True)
X['Severity'] = y

X['Weather_Timestamp'] = pd.to_datetime(X['Weather_Timestamp'])
X['day_of_week'] = X['Weather_Timestamp'].dt.day_name()

def is_weekday(day):
    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    return day in weekdays

# Apply the function to the day_of_week column to create the Weekday column
X['Weekday'] = X['day_of_week'].apply(lambda x: is_weekday(x))

X.drop(columns=['Weather_Timestamp', 'day_of_week'], inplace = True)

FileNotFoundError: [Errno 2] No such file or directory: 'US_Accidents_March23.csv'

In [ ]:


# Calculate the average severity for accidents in each city
city_severity = X.groupby('City')['Severity'].mean()

# Invert the severity to get a safety score (lower severity = higher safety)
max_severity = city_severity.max()
city_safety_score = max_severity - city_severity


In [ ]:
city_safety_score

City
Acworth         1.494460
Adairsville     1.494340
Adel            1.387755
Adrian          2.000000
Ailey           0.947368
                  ...   
Wray            0.714286
Wrens           1.000000
Wrightsville    1.125000
Young Harris    1.333333
Zebulon         1.452830
Name: Severity, Length: 525, dtype: float64

In [ ]:
import requests
import csv
from itertools import combinations
import pandas as pd  # Assuming pandas is used for city_safety_score calculation

# Load or calculate your city_safety_score here
# Example placeholder: city_safety_score = {'Atlanta': 2.5, 'Augusta-Richmond County (balance)': 3.0}
#city_safety_score = {}  # You need to fill this based on your accident data

def get_osrm_route(server_url, start, end, start_city, end_city):
    """ Get the route from the OSRM server and fetch the safety score. """
    try:
        request_url = f"{server_url}/route/v1/driving/{start[1]},{start[0]};{end[1]},{end[0]}?overview=full&geometries=geojson"
        response = requests.get(request_url)
        response.raise_for_status()
        data = response.json()
        route_geometry = data['routes'][0]['geometry']
        safety_score = (city_safety_score.get(start_city, 0) + city_safety_score.get(end_city, 0)) / 2
        return route_geometry, safety_score
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None, None

def save_routes_to_csv(routes, filename='routes.csv'):
    """ Save the routes data to a CSV file. """
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Origin', 'Destination', 'Safety Score', 'Route Geometry'])
        for route in routes:
            writer.writerow(route)

# Dictionary of cities and their coordinates
georgia_cities = {
    'Atlanta': (33.7629, -84.4227),
    'Augusta-Richmond County (balance)': (33.3655, -82.0734),
    'Columbus': (32.5102, -84.8749),
    'Macon-Bibb County': (32.8088, -83.6942),
    'Savannah': (32.018, -81.1965),
    'Athens-Clarke County (balance)': (33.9496, -83.3701),
    'South Fulton': (33.6357, -84.5834),
    'Sandy Springs': (33.9315, -84.3689),
    'Roswell': (34.0395, -84.3509),
    'Warner Robins': (32.5971, -83.6539),
    'Johns Creek': (34.0327, -84.2038),
    'Alpharetta': (34.0709, -84.2727),
    'Albany': (31.5781, -84.1762),
    'Marietta': (33.9536, -84.5425),
    'Stonecrest': (33.6807, -84.1452),
    'Brookhaven': (33.8612, -84.3385),
    'Smyrna': (33.8626, -84.5161),
    'Valdosta': (30.8501, -83.2788),
    'Dunwoody': (33.9414, -84.3127),
    'Gainesville': (34.2901, -83.8291)
}

# OSRM server URL
osrm_server = 'http://router.project-osrm.org'

# Create combinations of cities
city_combinations = combinations(georgia_cities.items(), 2)
all_routes = []

# Retrieve and store routes for each pair of cities
for (origin_name, origin_coords), (destination_name, destination_coords) in city_combinations:
    route_geometry, safety_score = get_osrm_route(osrm_server, origin_coords, destination_coords, origin_name, destination_name)
    if route_geometry:
        all_routes.append([origin_name, destination_name, safety_score, str(route_geometry)])
        print(f"Route from {origin_name} to {destination_name} with safety score {safety_score} obtained.")
    else:
        print("No route found.")
    print("---")

# Save all routes to a CSV file
save_routes_to_csv(all_routes)


In [ ]:
import requests
import csv
from itertools import combinations
import pandas as pd  # Ensure pandas is imported
from google.colab import files  # For downloading files in Google Colab

def get_osrm_route(server_url, start, end, start_city, end_city):
    try:
        request_url = f"http://router.project-osrm.org/route/v1/driving/{start[1]},{start[0]};{end[1]},{end[0]}?overview=full&geometries=geojson"
        response = requests.get(request_url)
        response.raise_for_status()
        data = response.json()
        route_geometry = data['routes'][0]['geometry']
        safety_score = (city_safety_score.get(start_city, 0) + city_safety_score.get(end_city, 0)) / 2
        return route_geometry, safety_score
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None, None

def save_routes_to_csv(routes, filename='routes.csv'):
    """ Save the routes data to a CSV file using pandas and download it in Google Colab. """
    # Convert routes to a DataFrame
    df = pd.DataFrame(routes, columns=['Origin', 'Destination', 'Safety Score', 'Route Geometry'])

    # Convert DataFrame to CSV string
    csv_string = df.to_csv(index=False)

    # Save to a CSV file and trigger download
    with open(filename, 'w') as f:
        f.write(csv_string)
    files.download(filename)  # Trigger the download

# Assuming city_safety_score is populated


# Rest of your script
# Dictionary of cities and their coordinates
georgia_cities = { 'Atlanta': (33.7629, -84.4227),
    'Augusta-Richmond County (balance)': (33.3655, -82.0734),
    'Columbus': (32.5102, -84.8749),
    'Macon-Bibb County': (32.8088, -83.6942),
    'Savannah': (32.018, -81.1965),
    'Athens-Clarke County (balance)': (33.9496, -83.3701),
    'South Fulton': (33.6357, -84.5834),
    'Sandy Springs': (33.9315, -84.3689),
    'Roswell': (34.0395, -84.3509),
    'Warner Robins': (32.5971, -83.6539),
    'Johns Creek': (34.0327, -84.2038),
    'Alpharetta': (34.0709, -84.2727),
    'Albany': (31.5781, -84.1762),
    'Marietta': (33.9536, -84.5425),
    'Stonecrest': (33.6807, -84.1452),
    'Brookhaven': (33.8612, -84.3385),
    'Smyrna': (33.8626, -84.5161),
    'Valdosta': (30.8501, -83.2788),
    'Dunwoody': (33.9414, -84.3127),
    'Gainesville': (34.2901, -83.8291)}  # your city list here

# OSRM server URL and other script logic
osrm_server = 'http://router.project-osrm.org'
city_combinations = combinations(georgia_cities.items(), 2)
all_routes = []

for (origin_name, origin_coords), (destination_name, destination_coords) in city_combinations:
    route_geometry, safety_score = get_osrm_route(osrm_server, origin_coords, destination_coords, origin_name, destination_name)
    if route_geometry:
        all_routes.append([origin_name, destination_name, safety_score, str(route_geometry)])
        print(f"Route from {origin_name} to {destination_name} with safety score {safety_score} obtained.")
    else:
        print("No route found.")
    print("---")

# Save all routes to a CSV file and download
save_routes_to_csv(all_routes)


Route from Atlanta to Augusta-Richmond County (balance) with safety score 0.7320773542271914 obtained.
---
Route from Atlanta to Columbus with safety score 1.4626403569081565 obtained.
---
Route from Atlanta to Macon-Bibb County with safety score 0.7320773542271914 obtained.
---
Route from Atlanta to Savannah with safety score 1.623659976414934 obtained.
---
Route from Atlanta to Athens-Clarke County (balance) with safety score 0.7320773542271914 obtained.
---
Route from Atlanta to South Fulton with safety score 0.7320773542271914 obtained.
---
Route from Atlanta to Sandy Springs with safety score 0.7320773542271914 obtained.
---
Route from Atlanta to Roswell with safety score 1.6100810510848624 obtained.
---
Route from Atlanta to Warner Robins with safety score 1.4415368136866509 obtained.
---
Route from Atlanta to Johns Creek with safety score 0.7320773542271914 obtained.
---
Route from Atlanta to Alpharetta with safety score 1.6754993037841104 obtained.
---
Route from Atlanta to Alb

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import json

def process_geometry(route_geometry):
    # Correct single quotes to double quotes and remove extra escape characters
    corrected_geometry = route_geometry.replace("'", '"').replace("\\", "")

    # Load the corrected GeoJSON LineString into a Python dictionary
    geometry = json.loads(corrected_geometry)
    coordinates = geometry.get('coordinates', [])

    # Create a list to hold our processed rows
    processed_rows = []
    for order, (longitude, latitude) in enumerate(coordinates):
        processed_rows.append({'Latitude': latitude, 'Longitude': longitude, 'Order': order})
    return processed_rows

# Assuming the 'Route Geometry' is in the correct column
# Let's test the function with a sample string that mimics your input data structure
sample_data = """{'type': 'LineString', 'coordinates': [[-84.422122, 33.7629], [-84.422122, 33.762835]]}"""

processed_rows = process_geometry(sample_data)
processed_df = pd.DataFrame(processed_rows)

# Display the processed DataFrame
print(processed_df)


    Latitude  Longitude  Order
0  33.762900 -84.422122      0
1  33.762835 -84.422122      1
